In [ ]:
# install bonita first
# !pip install -e git+https://github.com/BatsResearch/bonito#egg=bonito

We can start by downloading all our documentation from our Gitbook docs located
at docs.zenml.io. We'll use the langchain scraper to make this easy.

In [1]:
from zenml.client import Client

artifact = Client().get_artifact_version('86ba966e-66d1-4c79-a464-8bfff65300a0')
loaded_artifact = artifact.load()

Could not import GCP service connector: No module named 'google.api_core'.
Could not import Azure service connector: No module named 'azure.identity'.
Could not import Kubernetes service connector: No module named 'kubernetes'.
Could not import HyperAI service connector: No module named 'paramiko'.


In [2]:
from rich import inspect

inspect(loaded_artifact[0])

╭─────────────────────────────── <class 'langchain_core.documents.base.Document'> ────────────────────────────────╮
│ Class for storing a piece of text and associated metadata.                                                      │
│                                                                                                                 │
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Document(                                                                                                   │ │
│ │ │   page_content='An end-to-end project\n\nPut your new knowledge in action with'+2253,                     │ │
│ │ │   metadata={'source': 'https://docs.zenml.io/user-guide/production-guide/end-to-end'}                     │ │
│ │ )                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
│ lc_attributes = {}                                                                                              │
│    lc_secrets = {}                                                                                              │
│      metadata = {'source': 'https://docs.zenml.io/user-guide/production-guide/end-to-end'}                      │
│  page_content = 'An end-to-end project\n\nPut your new knowledge in action with an end-to-end project\n\nThat   │
│                 was awesome! We learned so many advanced MLOps production concepts:\n\nThe value of deploying   │
│                 ZenML\u200b\n\nAbstracting infrastructure configuration into stacks\u200b\n\n\u200bConnecting   │
│                 remote storage\u200b\n\n\u200bOrchestrating on the cloud\u200b\n\n\u200bConfiguring the         │
│                 pipeline to scale compute\u200b\n\n\u200bConnecting a git repository\u200b\n\nWe will now       │
│                 combine all of these concepts into an end-to-end MLOps project powered by ZenML.\n\nGet         │
│                 started\n\nStart with a fresh virtual environment with no dependencies. Then let\'s install our │
│                 dependencies:\n\npip install "zenml[templates,server]" notebook\n\nzenml integration install    │
│                 sklearn -y\n\nWe will then use\n\nZenML templates\n\nto help us get the code we need for the    │
│                 project:\n\nmkdir zenml_batch_e2e\n\ncd zenml_batch_e2e\n\nzenml init --template e2e_batch      │
│                 --template-with-defaults\n\n# Just in case, we install the requirements again\n\npip install -r │
│                 requirements.txt\n\nAbove doesn\'t work? Here is an alternative\n\nThe e2e template is also     │
│                 available as a\n\nZenML example\n\n. You can clone it:\n\ngit clone                             │
│                 [email\xa0protected]:zenml-io/zenml.git\n\ncd examples/e2e\n\npip install -r                    │
│                 requirements.txt\n\nzenml init\n\nWhat you\'ll learn\n\nThe e2e project is a comprehensive      │
│                 project template to cover major use cases of ZenML: a collection of steps and pipelines and, to │
│                 top it all off, a simple but useful CLI. It showcases the core ZenML concepts for supervised ML │
│                 with batch predictions. It builds on top of the\n\nstarter project\n\nwith more advanced        │
│                 concepts.\n\nAs you progress through the e2e batch template, try running the pipelines on       │
│                 a\n\nremote cloud stack\n\non a tracked\n\ngit repository\n\nto practice some of the concepts   │
│                 we have learned in this guide.\n\nAt the end, don\'t forget to share the\n\nZenML e2e           │
│                 template\n\nwith your colleagues and s

In [3]:
# !python -m spacy download en_core_web_sm

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")  # Load the English language model

def split_into_sentences(text):
    doc = nlp(text)  # Process the text with SpaCy
    sentences = [sent.text.strip() for sent in doc.sents]  # Extract sentences and strip whitespace
    return sentences

def process_documents(documents):
    all_sentences = []
    for doc in documents:
        text = doc.page_content  # Extract the text from the Document object
        sentences = split_into_sentences(text)  # Split the text into sentences
        all_sentences.extend(sentences)  # Add the sentences to the list
    return all_sentences

  # Your list of langchain Document objects
sentences = process_documents(loaded_artifact)  # Process the documents and get all sentences
sentences[0:5]

['An end-to-end project\n\nPut your new knowledge in action with an end-to-end project\n\nThat was awesome!',
 'We learned so many advanced MLOps production concepts:\n\nThe value of deploying ZenML\u200b\n\nAbstracting infrastructure configuration into stacks\u200b\n\n\u200bConnecting remote storage\u200b\n\n\u200bOrchestrating on the cloud\u200b\n\n\u200bConfiguring the pipeline to scale compute\u200b\n\n\u200bConnecting a git repository\u200b\n\nWe will now combine all of these concepts into an end-to-end MLOps project powered by ZenML.',
 'Get started\n\nStart with a fresh virtual environment with no dependencies.',
 'Then let\'s install our dependencies:\n\npip install "zenml[templates,server]" notebook\n\nzenml integration install sklearn -y\n\nWe will then use\n\nZenML templates\n\nto help us get the code we need for the project:',
 "mkdir zenml_batch_e2e\n\ncd zenml_batch_e2e\n\nzenml init --template e2e_batch --template-with-defaults\n\n# Just in case, we install the requireme

In [5]:
from datasets import Dataset

# Assuming sentences is a list of strings, where each string is a sentence
data = {"sentence": sentences}
dataset = Dataset.from_dict(data)

print(dataset)

PyTorch version 2.1.2 available.
Dataset({
    features: ['sentence'],
    num_rows: 8192
})


In [6]:
# !git clone https://github.com/BatsResearch/bonito.git
# %pip install -e bonito

In [7]:
from bonito import Bonito, SamplingParams
from datasets import load_dataset

# Initialize the Bonito model
bonito = Bonito("BatsResearch/bonito-v1", max_model_len=10000)

sampling_params = SamplingParams(max_tokens=256, top_p=0.95, temperature=0.5, n=1)
synthetic_dataset = bonito.generate_tasks(
    dataset,
    context_col="sentence",
    task_type="exqa",
    sampling_params=sampling_params
)

INFO 03-19 18:21:45 llm_engine.py:70] Initializing an LLM engine with config: model='BatsResearch/bonito-v1', tokenizer='BatsResearch/bonito-v1', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=10000, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 03-19 18:21:52 llm_engine.py:275] # GPU blocks: 3023, # CPU blocks: 2048
INFO 03-19 18:21:54 model_runner.py:501] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-19 18:21:54 model_runner.py:505] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode.
INFO 03-19 18:22:00 model_runner.py:547] Graph capturing finished in 6 secs.


Map:   0%|          | 0/8192 [00:00<?, ? examples/s]

Processed prompts: 100%|██████████| 8192/8192 [02:59<00:00, 45.68it/s] 


Filter:   0%|          | 0/8192 [00:00<?, ? examples/s]

Map:   0%|          | 0/8155 [00:00<?, ? examples/s]

In [8]:
synthetic_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 8155
})

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from huggingface_hub import create_repo
from huggingface_hub import Repository

repo_name = "synth-test-zenml-exqa"
repo_url = create_repo(repo_name, repo_type="dataset")
print("Repository URL:", repo_url)
synthetic_dataset.push_to_hub(f"strickvl/{repo_name}")

Repository URL: https://huggingface.co/datasets/strickvl/synth-test-zenml-exqa


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/strickvl/synth-test-zenml-exqa/commit/4d5602177c09744c7395eb0d87e53f4333d269b7', commit_message='Upload dataset', commit_description='', oid='4d5602177c09744c7395eb0d87e53f4333d269b7', pr_url=None, pr_revision=None, pr_num=None)